# Resolviendo EDOs con Python

**Nombre:** Heriberto Espino Montelongo

**Materia:** Análisis numérico

**Sección:** 1

**Fecha:** 07/11/2024

# Instrucciones

Considerando el problema de valor inicial (PVI):

Resolver:
$$
\frac{dy}{dt} = f(t, y)
$$
Sujeta a:
$$
y(t_0) = y_0
$$

# Funciones

In [1]:
import matplotlib.pyplot as plt
import mpld3
from IPython.display import display, HTML

import numpy as np
import sympy as sp
from sympy.abc import x 

from matplotlib.backends.backend_pdf import PdfPages

from prettytable import PrettyTable

In [2]:
all_figures = []

In [3]:
def plot_solution(t, y, fig_title, color):

    global all_figures  # Use global to store figures across multiple calls

    fig = plt.figure(figsize=(8, 6), dpi=1920) 
    plt.plot(t, y,'o', alpha=0.9, color=color)
    plt.title(fig_title, fontsize=14, fontweight='bold')
    plt.grid()
    plt.xlabel('t')
    plt.ylabel('y')

    display(HTML(mpld3.fig_to_html(plt.gcf()))) # Display the plot as HTML
    plt.close() # Close the plot to prevent it from displaying twice

    all_figures.append(fig) # Store the figure to save it later
    
    return

In [4]:
def exact_solution(name, function, lower_bound, upper_bound, h, plot = False):

    f = sp.lambdify(x,function)
    n = int((upper_bound-lower_bound)/h)
    t = np.linspace(lower_bound,upper_bound,n+1)
    y = f(t)

    if plot:
        plot_solution(t, y, 'Heri - Exact Solution - ' + str(name), '#B65A5A')

    return t, y

In [5]:
def euler(name, f, t0, tn, h, y0, plot = False):

    n = int(abs(tn - t0) / h)
    t = np.linspace(t0, tn, n + 1)
    y = np.zeros(n + 1)
    y[0] = y0
    for k in range(n):
        y[k + 1] = y[k] + h * f(t[k], y[k])

    if plot:
        plot_solution(t, y, 'Heri - Euler\'s method - ' + str(name), '#1A5465')

    return t, y

In [6]:
def RK4(name, f, t0, tn, h, y0, plot = False):

    n = int(abs(tn - t0) / h)
    t = np.linspace(t0, tn, n + 1)
    y = np.zeros(n + 1)
    y[0] = y0
    
    for i in range(n):
        s1 = f(t[i], y[i])
        s2 = f(t[i] + h / 2, y[i] + s1 * h / 2)
        s3 = f(t[i] + h / 2, y[i] + s2 * h / 2)
        s4 = f(t[i] + h, y[i] + s3 * h)
        y[i + 1] = y[i] + h * (s1 + 2 * s2 + 2 * s3 + s4) / 6

    if plot:
        plot_solution(t, y, 'Heri - RK4 method - ' + str(name), '#1A5465')

    return t, y

In [7]:
def compare_methods(name, function, f, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, function, t0, tn, h)
    t_euler, y_euler = euler(name, f, t0, tn, h, y0)
    t_rk4, y_rk4 = RK4(name, f, t0, tn, h, y0)

    global all_figures

    fig = plt.figure(figsize=(5, 3), dpi=150) 
    plt.plot(t_exact, y_exact, '-', label=f"Exact solution: {y_exact[-1]:.2f}", color='#50A3A4', alpha=0.5, lw = 4)
    plt.plot(t_euler, y_euler, 's--', label=f"Euler's Method: {y_euler[-1]:.2f}", color='#FCAF38', alpha=0.5)
    plt.plot(t_rk4, y_rk4, 'o--', label=f"RK4 Method: {y_rk4[-1]:.2f}", color='#F95335', alpha=0.5)
    plt.title('Comparison of Methods - ' + str(name), fontsize=14, fontweight='bold')
    plt.xlabel('t')
    plt.ylabel('y')
    plt.legend()
    plt.grid()

    display(HTML(mpld3.fig_to_html(plt.gcf())))
    plt.close()

    all_figures.append(fig)
    
    return

In [8]:
def euler_vs_RK4(name, f, t0, tn, h, y0, plot = False, table = False, final_results = False):

    t_euler, y_euler = euler(name, f, t0, tn, h, y0)
    t_rk4, y_rk4 = RK4(name, f, t0, tn, h, y0)

    if table:
        myTable = PrettyTable(["x_i", "y_i Euler", "y_i RK4"])
        myTable.title = "Euler vs RK4 " + str(name)

        for i in range(len(t_euler)):
            myTable.add_row([
                round(t_euler[i], 2), 
                round(y_euler[i], 2), 
                round(y_rk4[i], 2), 
            ])

        display(HTML(myTable.get_html_string()))

    if plot:
        global all_figures

        fig = plt.figure(figsize=(5, 3), dpi=150) 
        plt.plot(t_euler, y_euler, 's-', label=f"Euler's Method: {y_euler[-1]:.2f}", color='#1A5465', alpha=0.5)
        plt.plot(t_rk4, y_rk4, 'd-', label=f'RK4 Method: {y_rk4[-1]:.2f}', color='#2A9D8F', alpha=0.5)
        plt.title('Euler vs RK4 - ' + str(name), fontsize=14, fontweight='bold')
        plt.xlabel('t')
        plt.ylabel('y')
        plt.legend()
        plt.grid()
        
        display(HTML(mpld3.fig_to_html(plt.gcf())))
        plt.close()

        all_figures.append(fig)

    return

In [9]:
def error_euler(name, f, function, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, f, t0, tn, h)
    t_euler, y_euler = euler(name, function, t0, tn, h, y0)
    
    myTable = PrettyTable(["x_i", "y_i Exact", "y_i Euler", "Absolute E", "Relative E"])
    myTable.title = "Error Analysis for Euler  " + str(name)

    for i in range(len(t_exact)):
        myTable.add_row([
            round(t_exact[i], 4), 
            round(y_exact[i], 4), 
            round(y_euler[i], 4), 
            round(abs(y_exact[i] - y_euler[i]), 4), 
            round(abs(y_exact[i] - y_euler[i]) / y_exact[i], 4)
        ])

    display(HTML(myTable.get_html_string()))

In [10]:
def error_rk4(name, f, function, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, f, t0, tn, h)
    t_rk4, y_rk4 = RK4(name, function, t0, tn, h, y0)

    myTable = PrettyTable(["x_i", "y_i Exact", "y_i RK4", "Absolute E", "Relative E"])
    myTable.title = "Error Analysis for RK4  " + str(name)

    for i in range(len(t_exact)):
        myTable.add_row([
            round(t_exact[i], 2), 
            round(y_exact[i], 2), 
            round(y_rk4[i], 2), 
            round(abs(y_exact[i] - y_rk4[i]), 4), 
            round(abs(y_exact[i] - y_rk4[i]) / y_rk4[i], 4)
        ])
        
    display(HTML(myTable.get_html_string()))

In [11]:
def multipage(filename, figs=None, dpi=150):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]  # Fallback: get all open figures
    for fig in figs:
        # Save each figure with tight layout and specified dpi to control size
        fig.savefig(pp, format='pdf', dpi=dpi, bbox_inches='tight')
    pp.close()

# Ejercicio 1

Si se invierten $10,000 dólares a una tasa de interés del 3% anual, capitalizada semestralmente, encuentra el valor de la inversión después del número dado de años. Primero utiliza el método de Euler y luego el método RK4. Grafica la solución numérica con ambos métodos. Superpone ambas curvas de solución en el mismo sistema de coordenadas. Si es posible, utiliza un color diferente para cada curva. Interpreta la solución.
   - a) 5 años 
   - b) 10 años 
   - c) 15 años

In [12]:
def i(t, y):
    return 0.03 * y

t0 = 0 
y0 = 10000
h = 0.5

### Para 5 años

In [13]:
tn = 5
name = f"Ejercicio 1 - Inversión a {tn} años - h = {h}"

In [14]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A 5 años, con una tasa de interés del 3% anual, capitalizada semestralmente, $10,000 dólares se transformaron en $11,605.41 con el método de Euler y a $11,618.34 con el método de Runge-Kutta.

### Para 10 años

In [15]:
tn = 10
name = f"Ejercicio 1 - Inversión a {tn} años - h = {h}"

In [16]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A 10 años, con una tasa de interés del 3% anual, capitalizada semestralmente, $10,000 dólares se transformaron en $13,468.55 con el método de Euler y a $13,498.59 con el método de Runge-Kutta.

### Para 15 años

In [17]:
tn = 15
name = f"Ejercicio 1 - Inversión a {tn} años - h = {h}"

In [18]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A 15 años, con una tasa de interés del 3% anual, capitalizada semestralmente, $10,000 dólares se transformaron en $15,630.80 con el método de Euler y a $15,683.12 con el método de Runge-Kutta.

# Ejercicio 2

Si se invierten 500 dólares a una tasa de interés del 3.75% anual, capitalizada trimestralmente, encuentra el valor de la inversión después del número dado de años. Primero utiliza el método de Euler y luego el método RK4. Grafica la solución numérica con ambos métodos. Superpone ambas curvas de solución en el mismo sistema de coordenadas. Si es posible, utiliza un color diferente para cada curva. Interpreta la solución.
   - a) 1 año 
   - b) 2 años 
   - c) 10 años

In [19]:
def i(t, y):
    return 0.0375 * y

t0 = 0 
y0 = 500
h = 0.25

### Para 1 año

In [20]:
tn = 1
name = f"Ejercicio 2 - Inversión a {tn} año - h = {h}"

In [21]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A un año, con una tasa de interés del 3.75% anual, capitalizada trimestralmente, $500 dólares se transformaron en $519.02 con el método de Euler y a $519.11 con el método de Runge-Kutta.

### Para 2 años

In [22]:
tn = 2
name = f"Ejercicio 2 - Inversión a {tn} años - h = {h}"

In [23]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A dos años, con una tasa de interés del 3.75% anual, capitalizada trimestralmente, $500 dólares se transformaron en $538.75 con el método de Euler y a $538.94 con el método de Runge-Kutta.

###  Para 10 años

In [24]:
tn = 10
name = f"Ejercicio 2 - Inversión a {tn} años - h = {h}"

In [25]:
euler_vs_RK4(name, i, t0, tn, h, y0, plot=True)

A diez años, con una tasa de interés del 3.75% anual, capitalizada trimestralmente, $500 dólares se transformaron en $726.23 con el método de Euler y a $726.50 con el método de Runge-Kutta.

# Ejercicio 3

Considera el modelo simple de combustión dado por 
   $$
   y' = y^2 - y^3, \quad y(0) = y_0, \quad 0 \leq t \leq \frac{2}{y_0}
   $$
   donde $y$ es el radio de la bola de llama. Utiliza RK4 y el método de Euler para investigar el modelo de la llama con estos parámetros: $y_0 = \frac{1}{1000}$ y $h = \frac{t_{final}}{2000}$. Grafica la solución numérica con ambos métodos numéricos. Superpone ambas curvas de solución en el mismo sistema de coordenadas. Si es posible, utiliza un color diferente para cada curva. ¿Para qué valor de $t$ alcanza el radio el valor de 0.90?

In [26]:
def combustion_model(t, y):
    return y**2 - y**3

y0 = 1 / 1000
t0 = 0
t_final = 2 / y0
h = t_final / 2000
name = "Ejercicio 3 - Modelo de combustión"

euler_vs_RK4(name, combustion_model, t0, t_final, h, y0, plot=True)

En el modelo de combustión simple, el radio de la llama llega a $0.90$ entre $1013$ y $1014$ con el método Euler, y en $1014$ con el método de RK4.


| x_i   | y_i Euler | y_i RK4 |
|-------|-----------|---------|
| 995.0 | 0.07      | 0.1     |
| 996.0 | 0.08      | 0.11    |
| 997.0 | 0.08      | 0.13    |
| 998.0 | 0.09      | 0.14    |
| 999.0 | 0.1       | 0.16    |
| 1000.0| 0.11      | 0.18    |
| 1001.0| 0.12      | 0.22    |
| 1002.0| 0.13      | 0.26    |
| 1003.0| 0.14      | 0.32    |
| 1004.0| 0.16      | 0.4     |
| 1005.0| 0.18      | 0.51    |
| 1006.0| 0.21      | 0.65    |
| 1007.0| 0.25      | 0.79    |
| 1008.0| 0.29      | 0.9     |
| 1009.0| 0.35      | 0.96    |
| 1010.0| 0.43      | 0.98    |
| 1011.0| 0.54      | 0.99    |
| 1012.0| 0.67      | 1.0     |
| 1013.0| 0.82      | 1.0     |
| 1014.0| 0.94      | 1.0     |
| 1015.0| 0.99      | 1.0     |
| 1016.0| 1.0       | 1.0     |


# Ejercicio 4

Considerando 
   $$
   y' = 2xy, \quad y(1) = 1
   $$
   calcula $y(1.5)$. Utiliza RK4 y el método de Euler para obtener una aproximación de cuatro decimales del valor indicado. Primero, utiliza $h = 0.1$ y luego utiliza $h = 0.05$. Encuentra una solución analítica explícita para cada problema de valor inicial y luego construye tablas para calcular el error de cada método y todos los valores de la partición. Interpreta la solución. Grafica la solución numérica con ambos métodos y la solución analítica. Superpone las tres curvas de solución en el mismo sistema de coordenadas.

### Solución de la EDO

[mathdf.com](https://mathdf.com/dif/es/#expr=y'%3D2xy&func=y&arg=x&vals=1%3B1)

$$y={e}^{{x}^{2}-1}, \text{   } y_{0}\left(1\right)=1$$

### Parámetros

In [27]:
def function(x, y):
    return 2 * x * y

f = 'exp(x**2-1)'

t0 = 1
y0 = 1
tn = 1.5

### Para $h = 0.1$

In [28]:
h = 0.1
name = 'Ejercicio 4 - h = 0.1'

In [29]:
compare_methods(name, f, function, t0, tn, h, y0)

In [30]:
error_euler(name, f, function, t0, tn, h, y0)

x_i,y_i Exact,y_i Euler,Absolute E,Relative E
1.0,1.0,1.0,0.0,0.0
1.1,1.2337,1.2,0.0337,0.0273
1.2,1.5527,1.464,0.0887,0.0571
1.3,1.9937,1.8154,0.1784,0.0895
1.4,2.6117,2.2874,0.3243,0.1242
1.5,3.4903,2.9278,0.5625,0.1612


In [31]:
error_rk4(name, f, function, t0, tn, h, y0)

x_i,y_i Exact,y_i RK4,Absolute E,Relative E
1.0,1.0,1.0,0.0,0.0
1.1,1.23,1.23,0.0,0.0
1.2,1.55,1.55,0.0,0.0
1.3,1.99,1.99,0.0,0.0
1.4,2.61,2.61,0.0001,0.0
1.5,3.49,3.49,0.0001,0.0


### Para $h = 0.05$

In [32]:
h = 0.05
name = 'Ejercicio 4 - h = 0.05'

In [33]:
compare_methods(name, f, function, t0, tn, h, y0)

In [34]:
error_euler(name, f, function, t0, tn, h, y0)

x_i,y_i Exact,y_i Euler,Absolute E,Relative E
1.0,1.0,1.0,0.0,0.0
1.05,1.1079,1.1,0.0079,0.0072
1.1,1.2337,1.2155,0.0182,0.0147
1.15,1.3806,1.3492,0.0314,0.0227
1.2,1.5527,1.5044,0.0483,0.0311
1.25,1.7551,1.6849,0.0702,0.04
1.3,1.9937,1.8955,0.0982,0.0493
1.35,2.2762,2.1419,0.1343,0.059
1.4,2.6117,2.4311,0.1806,0.0692
1.45,3.0117,2.7714,0.2403,0.0798


In [35]:
error_rk4(name, f, function, t0, tn, h, y0)

x_i,y_i Exact,y_i RK4,Absolute E,Relative E
1.0,1.0,1.0,0.0,0.0
1.05,1.11,1.11,0.0,0.0
1.1,1.23,1.23,0.0,0.0
1.15,1.38,1.38,0.0,0.0
1.2,1.55,1.55,0.0,0.0
1.25,1.76,1.76,0.0,0.0
1.3,1.99,1.99,0.0,0.0
1.35,2.28,2.28,0.0,0.0
1.4,2.61,2.61,0.0,0.0
1.45,3.01,3.01,0.0,0.0


Concluimos que el método de Runge-Kutta es más cercano al valor verdadero que el método de Euler.

# Ejercicio 5

Considerando 
   $$
   y' = y(10 - 2y), \quad y(0) = 1
   $$
   obtén la solución numérica para el problema de valor inicial $IVP$ dado. Primero, utiliza el método de Euler y luego el método RK4. Utiliza $h = 0.25$ en cada caso. Superpone ambas curvas de solución en el mismo sistema de coordenadas. Si es posible, utiliza un color diferente para cada curva. Repite, utilizando $h = 0.1$ y $h = 0.05$.

### Solución de la EDO:

[mathdf.com](https://mathdf.com/dif/es/#expr=y'%3Dy(10-2y)&func=y&arg=x&vals=0%3B1)

$$
y=\dfrac{5}{-\dfrac{{e}^{10\,x}}{4}-1}+5, \text{   } y_{0}\left(0\right)=1
$$

### Parámetros

In [36]:
def growth_model(t, y):
    return y * (10 - 2 * y)

f = ' 5 / (-(exp(10 * x) / 4) - 1) + 5'

t0 = 0 
y0 = 1
tn = 5

### Para $h=0.25$

In [37]:
name = 'h = 0.25'
h = 0.25

In [38]:
compare_methods(name, f, growth_model, t0, tn, h, y0)

### Para $h=0.10$

In [39]:
name = 'h = 0.10'
h = 0.10

In [40]:
compare_methods(name, f, growth_model, t0, tn, 0.10, y0)

### Para $h=0.05$

In [41]:
name = 'h = 0.05'
h = 0.05

In [42]:
compare_methods(name, f, growth_model, t0, tn, 0.05, y0)

Concluimos que mientras $h$ sea menor, tendremos una mejor aproximación.

# Exportar los plots

In [43]:
# multipage("Practica 4.pdf", figs=all_figures)